In [1]:
import torch
from torchvision import models

print("Torch version:", torch.__version__)

model = models.resnet18(pretrained=True)
print("Model loaded successfully")

Torch version: 2.9.1+cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\anish/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


c:\Users\anish\Desktop\Project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\anish\Desktop\Project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100.0%


Model loaded successfully


In [2]:
import torch
import torch.nn as nn
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = models.resnet18(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for 4 classes
model.fc = nn.Linear(model.fc.in_features, 4)

model = model.to(device)

print("Model ready")

Using device: cpu
Model ready


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transforms (simple, CPU-safe)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load training dataset
train_dataset = datasets.ImageFolder(
    "Combined Dataset/train",
    transform=train_transform
)

print("Number of training images:", len(train_dataset))
print("Class mapping:", train_dataset.class_to_idx)

Number of training images: 10240
Class mapping: {'Mild Impairment': 0, 'Moderate Impairment': 1, 'No Impairment': 2, 'Very Mild Impairment': 3}


In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=8,      # CPU-safe
    shuffle=True
)

# Sanity check: load one batch
images, labels = next(iter(train_loader))

print("Batch image shape:", images.shape)
print("Batch labels:", labels)

Batch image shape: torch.Size([8, 3, 224, 224])
Batch labels: tensor([3, 0, 0, 1, 2, 2, 1, 3])


In [5]:
import torch.nn as nn
import torch.optim as optim
import torch

# Compute class counts from the dataset
class_counts = torch.tensor([
    sum(1 for _, y in train_dataset.samples if y == i)
    for i in range(4)
], dtype=torch.float)

print("Class counts:", class_counts.tolist())

# Inverse-frequency weights (to reduce bias)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum()

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

print("Loss and optimizer ready")

Class counts: [2560.0, 2560.0, 2560.0, 2560.0]
Loss and optimizer ready


In [6]:
model.train()

total_loss = 0.0

for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

avg_loss = total_loss / len(train_loader)
print("Epoch 1 - Training loss:", avg_loss)

Epoch 1 - Training loss: 1.120063592446968


In [ ]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Training loss: {avg_loss:.4f}")

Epoch 1 - Training loss: 0.8674
Epoch 2 - Training loss: 0.7903
Epoch 3 - Training loss: 0.7495


In [8]:
import torch

torch.save(model.state_dict(), "alz_resnet18.pt")
print("Model saved as alz_resnet18.pt")

Model saved as alz_resnet18.pt


In [1]:
import torch
import torch.nn as nn
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = models.resnet18(pretrained=True)

# Freeze everything first
for param in model.parameters():
    param.requires_grad = False

# 🔥 Unfreeze LAST TWO blocks
for param in model.layer3.parameters():
    param.requires_grad = True

for param in model.layer4.parameters():
    param.requires_grad = True

# Replace classifier
model.fc = nn.Linear(model.fc.in_features, 4)

model = model.to(device)
print("Model setup complete")

Using device: cpu


c:\Users\anish\Desktop\Project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\anish\Desktop\Project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model setup complete


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder(
    "Combined Dataset/train",
    transform=train_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True
)

print("Images:", len(train_dataset))
print("Classes:", train_dataset.class_to_idx)

Images: 10240
Classes: {'Mild Impairment': 0, 'Moderate Impairment': 1, 'No Impairment': 2, 'Very Mild Impairment': 3}


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

# 🔥 LOWER LR to prevent collapse
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)

print("Loss + optimizer ready")

Loss + optimizer ready


In [5]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

Epoch 1/5 - Loss: 0.5814
Epoch 2/5 - Loss: 0.3007
Epoch 3/5 - Loss: 0.1490
Epoch 4/5 - Loss: 0.0802
Epoch 5/5 - Loss: 0.0469


In [6]:
torch.save(model.state_dict(), "alz_resnet18.pt")
print("Model saved")

Model saved
